In [1]:
from api_circle import CircleAPI
from api_circle import render_json
from api_intercom import IntercomAPI
from migrate_circle_posts_to_intercom import MigrateCirclePostsToIntercom
import html
import pandas as pd

## Next

In [ ]:
migration = MigrateCirclePostsToIntercom()

In [ ]:
out = migration.migrate_all_articles(display_progress=True)

In [ ]:
circle_posts = CircleAPI.api_get_posts(render=False)
print(len(circle_posts))

In [ ]:
[p for p in circle_posts if 'Intégrer' in p['name']][0]['body']['body']

In [ ]:
[o for o in out if isinstance(o, dict) and 'loom' in o['body'].lower()][2]

In [ ]:
out

In [ ]:
# Comment créer et paramétrer un projet ?
# 

In [ ]:
render_json([p for p in circle_posts if p['name'] == "Comment créer et paramétrer un projet ?"])

In [ ]:
circle_posts[1]["body"]["body"]

In [ ]:
render_json(circle_posts[0])

In [ ]:
[p['name'] for p in circle_posts if 'Animation' in p['space_name']]

In [ ]:
res = migration.migrate_all_articles(display_progress=True)

In [ ]:
render_json(res)

In [ ]:
res[-1]["body"]

In [ ]:
_ = IntercomAPI.api_get_collections()

In [ ]:
_ = IntercomAPI.api_get_help_centers()

In [ ]:
_ = IntercomAPI.api_get_teams()

In [ ]:
_ = IntercomAPI.api_get_articles()

In [ ]:
#IntercomAPI.api_create_article(title='BOBOBO', author_id="6624076", body="")

In [ ]:
from migrate_circle_posts_to_intercom import MigrateCirclePostsToIntercom
migration = MigrateCirclePostsToIntercom()

In [ ]:
#migration.delete_all_intercom_collections(are_you_sure=False)

In [ ]:
migration.migrate_all_collections()

## Circle (misc)

In [ ]:
#me = CircleAPI.api_get_me()

In [ ]:
#communities = CircleAPI.api_get_communities()
#print(f"nb communities: {len(communities)}")

## delete intercom collections

In [ ]:
# collections_id = [e['id'] for e in IntercomAPI.api_get_collections(render=False)['data'] if e['parent_id'] is None]
# for coll_id in collections_id:
#    IntercomAPI.api_delete_collection(coll_id=coll_id, render=False)

## Circle

In [ ]:
render=False
space_groups = CircleAPI.api_get_space_groups(render=render)
spaces = CircleAPI.api_get_spaces(render=render)
print(f"nb space groups: {len(space_groups)}")
print(f"nb spaces: {len(spaces)}")

In [ ]:
df_collections_lookup = pd.DataFrame(
    {'name': [s['name'] for s in space_groups] + [s['name'] for s in spaces],
     'parent_name': [None]*len(space_groups) + [s['space_group_name'] for s in spaces],
     'circle_id': [s['id'] for s in space_groups] + [str(s['id']) for s in spaces],
     'circle_parent_id': [None]*len(space_groups) + [str(s['space_group_id']) for s in spaces],
     'intercom_id': None,
     'intercom_parent_id': None}).sort_values('parent_name', ascending=True, na_position='first').set_index('name')

In [ ]:
## METHOD 1: import everything, whatever already exists in Intercom
root_collections = df_collections_lookup.index[df_collections_lookup['parent_name'].apply(lambda x: x is None)]
intercom_collections_created = [IntercomAPI.api_create_collection(n, render=False) for n in root_collections]

## add intercom IDs to lookup table
idx = [html.unescape(e['name']) for e in intercom_collections_created]
intercom_id = [e['id'] for e in intercom_collections_created]
df_collections_lookup.loc[idx, 'intercom_id'] = intercom_id
## add parent ID for intercom sub collections
for i in range(len(df_collections_lookup)):
    coll_name = df_collections_lookup.index[i]
    parent_name = df_collections_lookup.iloc[i]['parent_name']
    if parent_name is not None:
        df_collections_lookup.loc[coll_name,'intercom_parent_id'] = df_collections_lookup.loc[parent_name,'intercom_id']

In [ ]:
## create sub collections
sub_collections_name = df_collections_lookup.index[df_collections_lookup['parent_name'].apply(lambda x: x is not None)]
sub_collections_parent_id = df_collections_lookup.loc[sub_collections_name, 'intercom_parent_id']
intercom_sub_collections_created = [IntercomAPI.api_create_collection(name=n, parent_id=i, render=False) for (n,i) in sub_collections_parent_id.items()]

## add intercom ID
for e in intercom_sub_collections_created:
    df_collections_lookup.loc[html.unescape(e['name']), 'intercom_id'] = e['id']

In [ ]:
df_collections_lookup

## migrate posts

In [ ]:
intercom_admins = IntercomAPI.api_get_admins()

In [ ]:
circle_members = CircleAPI.api_get_members()

In [ ]:
[m for m in circle_members if 'Camila' in m['name']]

In [ ]:
circle_posts = CircleAPI.api_get_posts(render=False)

In [ ]:
circle_authors = pd.Series([p['user_name'] for p in circle_posts]).unique()
circle_authors

In [ ]:
intercom_admins

In [ ]:
# ## METHOD 2: Do not add collections already created
# root_collections = df_collections_lookup.index[df_collections_lookup['parent_name'].apply(lambda x: x is None)]
# ## get ID of collections already created in Intercom
# existing_intercom_collections = IntercomAPI.api_get_collections(render=False)['data']
# existing_intercom_collections_name = [e['name'] for e in existing_intercom_collections if e['name'] in df_collections_lookup.index]
# existing_intercom_collections_id = [e['id'] for e in existing_intercom_collections if e['name'] in df_collections_lookup.index]
# if len(existing_intercom_collections_name) > 0:
#     df_collections_lookup.loc[existing_intercom_collections_name, 'intercom_id'] = existing_intercom_collections_id

# ## create new Intercom root collections
# intercom_root_collections_to_create = [df_collections_lookup.index[i] for i in range(len(df_collections_lookup)) if (df_collections_lookup.iloc[i]['intercom_id'] is None) and (df_collections_lookup.index[i] in root_collections)]
# intercom_collections_created = [IntercomAPI.api_create_collection(n, render=False) for n in intercom_root_collections_to_create]

# ## add Ids to lookup table
# idx = [html.unescape(e['name']) for e in intercom_collections_created]
# intercom_id = [e['id'] for e in intercom_collections_created]
# df_collections_lookup.loc[idx, 'intercom_id'] = intercom_id

# ## add parent ID for intercom sub collections
# for i in range(len(df_collections_lookup)):
#     coll_name = df_collections_lookup.index[i]
#     parent_name = df_collections_lookup.iloc[i]['parent_name']
#     if parent_name is not None:
#         df_collections_lookup.loc[coll_name,'intercom_parent_id'] = df_collections_lookup.loc[parent_name,'intercom_id']

## Posts

In [ ]:
from migrate_circle_posts_to_intercom import MigrateCirclePostsToIntercom
migration = MigrateCirclePostsToIntercom()

In [ ]:
intercom_admins = IntercomAPI.api_get_admins(render=False)["admins"]
intercom_admins_id_lookup = {admin['name']:admin['id'] for admin in intercom_admins}

In [ ]:
intercom_admins_id_lookup

In [ ]:
posts = CircleAPI.api_get_posts(render=False)
print(f"nb posts: {len(posts)}")

In [ ]:
pd.Series([p['body']['record_type'] for p in posts]).value_counts()

In [ ]:
DEFAULT_INTERCOM_AUTHOR = 'Support technique'
p = posts[0]
r = IntercomAPI.api_create_article(
    title = p['name'],
    body= p['body']['body'],
    author_id= intercom_admins_id_lookup.get(p['user_name'], intercom_admins_id_lookup[DEFAULT_INTERCOM_AUTHOR]),
    state='published',
    parent_id = migration.df_collections_lookup.loc[p['space_name'], 'intercom_id']
)

In [ ]:
intercom_admins_id_lookup.get('Camila Grossi', intercom_admins_id_lookup[DEFAULT_INTERCOM_AUTHOR])

In [ ]:
render_json(posts[0])

In [ ]:
intercom_admins_id_lookup

In [ ]:
colls = IntercomAPI.api_get_collections(render=False)['data']
colls_name = [c['name'] for c in colls if c['name'] in migration.df_collections_lookup.index]
colls_idx = [c['id'] for c in colls if c['name'] in migration.df_collections_lookup.index]

## set intercom id
migration.df_collections_lookup.loc[colls_name, 'intercom_id'] = colls_idx

## set intercom parent id
for i in range(len(migration.df_collections_lookup)):
    coll_name = migration.df_collections_lookup.index[i]
    parent_name = migration.df_collections_lookup.iloc[i]["parent_name"]
    if parent_name is not None:
        migration.df_collections_lookup.loc[coll_name, "intercom_parent_id"] = (
            migration.df_collections_lookup.loc[parent_name, "intercom_id"]
        )

In [ ]:
migration.df_collections_lookup

In [ ]:
import random
i = random.randint(0, len(posts))
for k in posts[i].keys():
    print(k)
print(len(posts[i].keys()))

In [ ]:
posts[0]

## Intercom

In [ ]:
intercom_collections = IntercomAPI.api_get_collections()

In [ ]:
res = IntercomAPI.api_create_collection(name="new collection")

In [ ]:
res = IntercomAPI.api_create_collection(name="aa", parent_id="8401934")

# Lookup table of articles - Circle vs Intercom

In [ ]:
posts = CircleAPI.api_get_posts(render=False)
print(len(posts))

df_posts_circle = pd.DataFrame([{'circle_title': p['name'], 'circle_url': p['url']} for p in posts])

In [ ]:
articles = IntercomAPI.api_get_articles(render=False)
print(len(articles['data']))

5


In [ ]:
df_articles_intercom = pd.DataFrame([{'intercom_title': a['title'], 'intercom_id': a['id'], 'intercom_url': a['url']} for a in articles['data']])

In [ ]:
df_articles = pd.merge(left = df_posts_circle, right = df_articles_intercom, left_on = 'circle_title', right_on='intercom_title', how = 'left')

In [ ]:
is_problem_article = (df_articles['circle_title'] != df_articles['intercom_title'])
print(is_problem_article.sum())
df_articles[is_problem_article]

11


,circle_title,circle_url,intercom_title,intercom_id,intercom_url
12,Comment créer et paramétrer un projet ?,https://academiedidask.circle.so/c/creation/cr...,NaN,NaN,NaN
21,📣 Nouveautés du 31 août 2023 : Les relances au...,https://academiedidask.circle.so/c/nouveautes-...,NaN,NaN,NaN
29,Comment verrouiller des modules pour les diffu...,https://academiedidask.circle.so/c/animation/v...,NaN,NaN,NaN
34,Exemples de questions pour faire évaluer votre...,https://academiedidask.circle.so/c/animation/e...,NaN,NaN,NaN
76,🔐 Documentation Technique Didask,https://academiedidask.circle.so/c/diffusion/d...,NaN,NaN,NaN
79,🧠 Ajouter des Flashcards de mémorisation,https://academiedidask.circle.so/c/creation/aj...,NaN,NaN,NaN
94,👁️ Choisir l’affichage de vos granules d’exer...,https://academiedidask.circle.so/c/creation/ch...,NaN,NaN,NaN
95,🎨 Les modes du Créateur Visuel,https://academiedidask.circle.so/c/creation/mo...,NaN,NaN,NaN
112,"🤷‍♀️ Personne ne s'inscrit, que faire ?",https://academiedidask.circle.so/c/animation/p...,NaN,NaN,NaN
116,✏️ Ajouter une description de publication,https://academiedidask.circle.so/c/publication...,NaN,NaN,NaN


In [ ]:
df_articles['intercom_title'].duplicated().sum()

10

In [ ]:
[a['title'] for a in articles['data'] if "projet"  in a['title']]

['🚀 Tout savoir sur la publication de vos projets',
 '🤝 Rendre un projet visible aux membres de mon équipe',
 '⤴️ Comment copier un granule, un module ou un projet vers un autre projet ?',
 "Quand proposer de l'apprentissage par projet ?",
 '📣 Nouveautés du 11 septembre 2023 : Paramétrer des avatars pour l’ensemble d’un projet + Filtrer les statistiques',
 '👨 Paramétrer des avatars dans un projet']